# Concurrent Collections - Public API Guide

This notebook demonstrates the public API (Tier 4) of `concurrent_collections`:
- Ordered Maps: `SkipListMap`, `TreeMap`, `FrozenSkipListMap`
- Ordered Sets: `SkipListSet`, `TreeSet`, `FrozenSkipListSet`
- Queues: `LockFreeQueue`, `FastQueue`, `WaitFreeQueue`
- Stack: `LockFreeStack`
- Extensions: `BoundedSkipListMap`

In [ ]:
import threading
import time
from concurrent.futures import ThreadPoolExecutor

# Try to import real library
try:
    from concurrent_collections import (
        SkipListMap, SkipListSet, TreeMap, TreeSet,
        FrozenSkipListMap, FrozenSkipListSet,
        LockFreeQueue, FastQueue, WaitFreeQueue,
        LockFreeStack, BoundedSkipListMap
    )
    SIMULATION_MODE = False
    print("Using concurrent_collections library")
except ImportError:
    SIMULATION_MODE = True
    print("Library not installed - examples will be descriptive")

## Ordered Maps

### SkipListMap - Primary Ordered Map

In [ ]:
if not SIMULATION_MODE:
    # Basic usage
    m = SkipListMap()
    m['alice'] = 100
    m['bob'] = 200
    m['carol'] = 300
    
    print("Basic operations:")
    print(f"  m['alice'] = {m['alice']}")
    print(f"  len(m) = {len(m)}")
    print(f"  'bob' in m = {'bob' in m}")
    
    # Ordered operations
    print("\nOrdered operations:")
    print(f"  first_key() = {m.first_key()}")
    print(f"  last_key() = {m.last_key()}")
    print(f"  floor_key('charlie') = {m.floor_key('charlie')}")
    print(f"  ceiling_key('aaron') = {m.ceiling_key('aaron')}")
    
    # Range iteration
    print("\nRange iteration [a, c):")
    for k, v in m.items('a', 'c'):
        print(f"  {k}: {v}")
else:
    print("""
    # SkipListMap Usage Example:
    
    m = SkipListMap()
    m['alice'] = 100
    m['bob'] = 200
    
    # Ordered operations
    m.first_key()      # 'alice'
    m.last_key()       # 'bob'
    m.floor_key('az')  # 'alice'
    
    # Range iteration
    for k in m.keys('a', 'c'):
        print(k)  # alice, bob
    """)

### Atomic Operations

In [ ]:
print("""
# Atomic operations for concurrent access:

# Put only if key doesn't exist
old = m.put_if_absent('key', value)
# Returns None if inserted, old value if existed

# Replace only if key exists
old = m.replace('key', new_value)
# Returns old value if replaced, None if key didn't exist

# Conditional replace
success = m.replace_if('key', expected_old, new_value)
# Returns True only if old value matched

# Compute if absent (lazy initialization)
value = m.compute_if_absent('key', lambda k: expensive_compute(k))
# Computes only if key doesn't exist
""")

### Concurrent Access

In [ ]:
if not SIMULATION_MODE:
    m = SkipListMap()
    
    def writer(base):
        for i in range(1000):
            m[f"{base}_{i:04d}"] = i
    
    # Concurrent writes
    threads = [threading.Thread(target=writer, args=(chr(65+i),)) for i in range(4)]
    [t.start() for t in threads]
    [t.join() for t in threads]
    
    print(f"Total entries: {len(m)}")
    print(f"First 5 keys: {list(m.keys())[:5]}")
    print(f"Last 5 keys: {list(m.keys())[-5:]}")
else:
    print("Concurrent writes are fully supported - all operations are thread-safe")

## Ordered Sets

In [ ]:
print("""
# SkipListSet - Ordered Set

s = SkipListSet([3, 1, 4, 1, 5, 9, 2, 6])
print(list(s))  # [1, 2, 3, 4, 5, 6, 9] - sorted, unique

# Ordered operations
s.first()      # 1
s.last()       # 9
s.floor(7)     # 6
s.ceiling(7)   # 9

# Range iteration
list(s.range(3, 7))  # [3, 4, 5, 6]

# Set operations
s2 = SkipListSet([5, 6, 7, 8])
s.union(s2)         # {1,2,3,4,5,6,7,8,9}
s.intersection(s2)  # {5, 6}
""")

## Frozen (Immutable) Collections

In [ ]:
print("""
# FrozenSkipListMap - Immutable ordered map

m = SkipListMap({'a': 1, 'b': 2})
frozen = m.snapshot()  # Create immutable snapshot

# Read operations work
print(frozen['a'])       # 1
print(frozen.first_key())  # 'a'

# Mutations raise TypeError
frozen['c'] = 3  # TypeError!

# Can be used as dict key (if contents hashable)
cache = {frozen: "cached_value"}

# Convert back to mutable
mutable = frozen.thaw()
mutable['c'] = 3  # OK
""")

## Concurrent Queues

In [ ]:
print("""
# Three queue implementations:

# 1. LockFreeQueue (SCQ) - Portable, good performance
q = LockFreeQueue(maxsize=1000)

# 2. FastQueue - Auto-selects best backend
q = FastQueue(maxsize=1000)  # LCRQ on x86-64, SCQ elsewhere

# 3. WaitFreeQueue - Bounded latency guarantee
q = WaitFreeQueue(maxsize=1000, max_threads=8)

# All have same API:
q.put(item)           # Block if full
q.put_nowait(item)    # Raise Full
q.try_put(item)       # Return False

item = q.get()        # Block if empty
item = q.get_nowait() # Raise Empty  
item = q.try_get()    # Return None

# Bulk drain
items = q.drain(100)  # Get up to 100 items
""")

## Lock-Free Stack

In [ ]:
print("""
# LockFreeStack with elimination backoff

stack = LockFreeStack()

# Push items
stack.push(1)
stack.push(2)
stack.push(3)

# Pop in LIFO order
print(stack.pop())  # 3
print(stack.pop())  # 2

# Peek without removing
print(stack.peek())  # 1

# Non-blocking pop
item = stack.try_pop()  # Returns None if empty

# Elimination optimization automatically engages
# under high contention for better performance
""")

## Bounded Collections

In [ ]:
print("""
# BoundedSkipListMap - Size-limited ordered map

# Fail on overflow
m = BoundedSkipListMap(maxsize=100, eviction='error')

# Evict oldest (smallest key)
cache = BoundedSkipListMap(maxsize=100, eviction='oldest')

# Evict newest (largest key)
window = BoundedSkipListMap(maxsize=100, eviction='newest')

# Check capacity
cache.is_full()           # True/False
cache.remaining_capacity()  # Number
cache.maxsize              # 100
""")

## Custom Comparators

In [ ]:
print("""
# Custom ordering with comparators

# Reverse order
m = SkipListMap(cmp=lambda a, b: b - a)

# Case-insensitive strings
m = SkipListMap(key=lambda x: x.lower())

# By object attribute
m = SkipListMap(key=lambda x: x.priority)

# Check comparator type
print(m.comparator_type)  # 'key_func', 'cmp_func', 'natural'
""")

## Backend Selection

In [ ]:
print("""
# Backend selection for maps/sets

# Automatic (recommended) - chooses based on GIL state
m = SkipListMap(backend='auto')

# Force lock-free (no-GIL Python only)
m = SkipListMap(backend='lockfree')

# Force locked (always works)
m = SkipListMap(backend='locked')

# Check which backend is active
print(m.backend_type)  # 'lockfree' or 'locked'
""")

## Selection Guide

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════════╗
║                    COLLECTION SELECTION GUIDE                      ║
╠════════════════════════════════════════════════════════════════════╣
║ Need                          │ Use                                ║
╠═══════════════════════════════╪════════════════════════════════════╣
║ Ordered key-value store       │ SkipListMap                        ║
║ Ordered unique values         │ SkipListSet                        ║
║ Range queries important       │ SkipListMap/Set (faster ranges)    ║
║ Memory constrained            │ TreeMap/Set (lower memory)         ║
║ Immutable snapshot            │ FrozenSkipListMap/Set              ║
║ FIFO queue, max throughput    │ FastQueue                          ║
║ FIFO queue, portable          │ LockFreeQueue                      ║
║ FIFO queue, bounded latency   │ WaitFreeQueue                      ║
║ LIFO stack                    │ LockFreeStack                      ║
║ Size-limited cache            │ BoundedSkipListMap                 ║
╚═══════════════════════════════╧════════════════════════════════════╝
""")